In [1]:
import os
import sys
import subprocess
root = "/data/luojaa/eukgen/"
%cd {root}
sys.path.append(root)
import pandas as pd
import altair as alt
#import altair_saver

import ete3
from ete3 import Tree, TreeStyle, TextFace

from core_functions.microcosm_functions import color_tree, tree_analysis
from core_functions.altair_plots import plot_alignment, plot_cumsum_counts
from core_functions.tree_functions import get_outlier_nodes_by_lognorm

#disable altair max rows
alt.data_transformers.disable_max_rows()
#get default altair style'
%run /data/luojaa/eukgen/styling_and_visualisation.py

#dont wrap text output from cells
from IPython.display import display, HTML
display(HTML("<style>div.output_area pre {white-space: pre;}</style>"))

/vf/users/luojaa/eukgen


In [27]:
os.environ['QT_QPA_PLATFORM']='offscreen'
pd.set_option('display.max_colwidth', None)

In [28]:
# a = pd.read_csv("/data/luojaa/kegg_hmm_db/search_asgardvkegg/search_results.cut.tsv", sep = "\t")
# a = a[a.Query != a.Target]

In [29]:
# b = a[(a.Prob > 80) & (a.Pairwise_cov > .5)]
# b.to_csv("/data/luojaa/kegg_hmm_db/search_asgardvkegg/search_results.filtered.tsv", sep = "\t", index = None)

### read in aggregate files

In [30]:
tree_data = pd.read_csv("/data/luojaa/eukgen/microcosm_asgard_final_tree_data.tsv", sep = "\t")
leaf_mapping = pd.read_csv("/data/luojaa/eukgen/microcosm_asgard_final_tree_data.leaf_mapping.tsv", sep = "\t", )
# header_mapping = pd.read_csv("/data/tobiassonva/data/eukgen/core_data/euk72_ep/euk72_ep.fasta.header_mapping", sep = "\t", header = None)
# header_mapping.rename(columns = {0: "acc", 1:"header"}, inplace = True)
# header_mapping.set_index("acc", inplace = True)

In [31]:
# not all microcosms are annotated, and not all annotated clusters made it to microcosms
kegg_annot = pd.read_csv("/data/luojaa/kegg_hmm_db/search_asgardvkegg/asgard2023.KEGG_annotation.swapped_names.tsv", sep = "\t")
kegg_annot["kogid"] = kegg_annot.Target.apply(lambda x: x.split("_")[0])
KOGs_desc = pd.read_csv("/data/luojaa/kegg_stats/KOG_merged.tsv", sep = "\t")
kegg_annot = pd.merge(kegg_annot, KOGs_desc, how = "left", on = "kogid")
annotated_clusters = set(kegg_annot.Query)

In [7]:
# # swap hh2mm names

# asgard_hh2mm = pd.read_csv("/data/luojaa/eukgen/microcosm/asgards_mm2hh_map.tsv", sep = "\t", header = None).set_index(1).to_dict()[0]
# kegg_annot["Query_mm"] = kegg_annot.Query.apply(lambda x: asgard_hh2mm[x])
# kegg_annot["Query"] = kegg_annot["Query_mm"]
# kegg_annot.drop("Query_mm", axis=1).to_csv("/data/luojaa/kegg_hmm_db/search_asgardvkegg/asgard2023.KEGG_annotation.swapped_names.tsv", sep = "\t", index = None)

In [246]:
len(tree_data_annot.tree_name.unique())

1787

In [32]:
# filter out unannotated microcosms
tree_data.set_index("tree_name", inplace = True)
tree_data_annot = tree_data[tree_data.index.isin(annotated_clusters)].reset_index()

In [254]:
def get_hgt(df):
    return df[((df['c-ELW'].between(0.4, 1)))
              & (df['deltaL'] != -1)
              & (df.euk_clade_size > 4)]

In [255]:
# map KOG annotations to tree data
tree_data_annot_tmp = tree_data_annot.set_index("tree_name")
kegg_annot_tmp = kegg_annot.set_index("Query")[["Target", "Score", "name", "Pairwise_cov", "kogid", "brite_A", "brite_B", "brite_C", "EC", "TC"]]
tree_data_mapped = pd.merge(tree_data_annot_tmp, kegg_annot_tmp, left_index= True, right_index= True, how= "left")#.drop("Unnamed: 0", axis = 1)
tree_data_tophit = tree_data_mapped.sort_values(["Score"], ascending = False).reset_index().drop_duplicates(["euk_clade_rep", "c-ELW"]).sort_values(["tree_name"], ascending = False)
tree_data_tophit = get_hgt(tree_data_tophit)
#tree_data_tophit = get_hgt(tree_data)
# tree_data_important = tree_data_tophit[tree_data_tophit.euk_scope_len > 5]
# tree_data_universal = tree_data_tophit[tree_data_tophit.euk_scope_len > 10]

In [256]:
tree_data_pvc = tree_data_tophit[tree_data_tophit.prok_taxa == "PVC group"]

In [257]:
tree_data_tophit.prok_taxa.value_counts()

prok_taxa
Euryarchaeota              192
TACK group                 169
Bacillota                  113
Actinomycetota              69
FCB group                   46
Gammaproteobacteria         29
Thermodesulfobacteriota     21
Cyanobacteriota             17
Thermoplasmata              16
Alphaproteobacteria         16
PVC group                   16
Betaproteobacteria           9
Thermotogae                  8
Mycoplasmatota               6
Spirochaetota                5
Chloroflexota                5
Deinococcota                 4
Campylobacterota             3
Myxococcota                  2
Acidobacteriota              2
Nitrospirota                 1
Bdellovibrionota             1
Name: count, dtype: int64

In [126]:
# read in the kegg maps and modules
kog_functions = pd.read_csv('/data/luojaa/uid_stats/kog_functions.tsv', sep = '\t').drop_duplicates("KOGID")
category_data = pd.read_csv('/data/luojaa/kegg_stats/kegg_categories.tsv', sep = '\t')
category_data = pd.merge(category_data, kog_functions, on = "KOGID", how = "outer").dropna()
module_data = category_data[category_data["CATEGORY_ID"].str.startswith("M")]
pathway_data = category_data[category_data["CATEGORY_ID"].str.contains("map")]


In [127]:
# read in BRITE data
kog_brite = pd.read_csv("/data/luojaa/kegg_stats/KOG_merged.tsv", sep = "\t")
brite_desc = pd.read_csv("/data/tobiassonva/data/databases/kegg/kegg_metadata/KOG_brite_map.tsv", dtype=str, sep = "\t").set_index("brite_id")
kogs_w_pathway = set(pathway_data.KOGID)
kog_brite_nomap = kog_brite[~kog_brite.kogid.isin(kogs_w_pathway)].reset_index()
kog_brite_nomap["brite"] = kog_brite_nomap.brite_C.apply(lambda x: x.split("|"))
kog_brite_nomap = kog_brite_nomap.explode("brite").reset_index()

In [128]:
# brite_counts = kog_brite.brite_C.apply(lambda x: x.split("|")).explode().value_counts()
# brite_nomap_counts = kog_brite_nomap.brite.value_counts()
# brite_nomap_fraction = pd.merge(brite_counts,brite_nomap_counts, how = "right", left_index=True, right_index=True)
# brite_nomap_fraction["fraction_missing"] = brite_nomap_fraction.count_y / brite_nomap_fraction.count_x
# brite_nomap_fraction = pd.merge(brite_nomap_fraction, brite_desc, how = "left", left_index=True, right_index=True)

In [129]:
def summarize(df, cELW = .5, cladesize = 5):
    pd.set_option('display.max_colwidth', None)
    vc = df.prok_taxa.value_counts()
    vc_cELW = df[df["c-ELW"] > cELW].prok_taxa.value_counts()
    vc_cladesize = df[df.euk_clade_size > cladesize].prok_taxa.value_counts()
    vc_filt = df[(df["c-ELW"] > cELW) & (df.euk_clade_size > cladesize)].prok_taxa.value_counts()
    outdf = pd.concat([vc, vc_cELW, vc_cladesize, vc_filt], axis = 1)
    outdf.columns = ["unfiltered", "cELW_filt", "cladesize_filt", "all_filt"]
    return outdf

In [65]:
# map_now = next(maps_iter)
# name, df = pathids[map_now], summarize(filtered_dfs[map_now], .3)
# print(name)
# df

## View microcosms

In [90]:
briteA_2name = brite_desc[brite_desc.level == "A"].set_index("brite_A")["name"].to_dict()
briteB_2name = brite_desc[brite_desc.level == "B"].set_index("brite_B")["name"].to_dict()
briteC_2name = brite_desc[brite_desc.level == "C"].set_index("brite_C")["name"].to_dict()
def map_entry(entry, dic):
    try:
        return dic[entry]
    except:
        try:
            return [dic[bid] for bid in entry.split("|")]
        except:
            return "None"
tree_data_tophit["desc"] = tree_data_tophit.brite_C.apply(lambda x: map_entry(x, briteC_2name))

In [43]:
#pd.set_option('display.max_colwidth', None)

In [44]:
tree_data_tophit_vc = tree_data_tophit.brite_C.value_counts().reset_index()
tree_data_tophit_vc["desc"] = tree_data_tophit_vc.brite_C.apply(lambda x: map_entry(x, briteC_2name))
briteC_counts = tree_data_tophit_vc.set_index("count")["desc"].reset_index().iloc[:50]
#briteC_counts[briteC_counts.desc != "None"]


In [45]:
tree_data_TC = tree_data_tophit[(tree_data_tophit.TC != "none")].reset_index()
tree_data_TC["desc"] = tree_data_TC.brite_C.apply(lambda x: map_entry(x, briteC_2name))
tree_data_TC_groups = tree_data_TC.explode("desc").groupby(["desc", "name"]).size().reset_index().sort_values(0, ascending = False)
#tree_data_TC.desc.explode().value_counts().reset_index()


In [46]:
tree_data_proksis = tree_data_tophit[(tree_data_tophit["c-ELW"] > .7) & (~tree_data_tophit.prok_taxa.isin(["Euryarchaeota", "TACK group", "Thermoplasmatota", "DPANN group"]))].reset_index().sort_values("kogid")
tree_data_proksis["desc"] = tree_data_proksis.brite_C.apply(lambda x: map_entry(x, briteC_2name))
proksis_desc_vc = tree_data_proksis.desc.explode().value_counts() 
tophit_desc_vc = tree_data_tophit[(tree_data_tophit["c-ELW"] > .3)].desc.explode().value_counts()
most_prok_briteC = pd.merge(proksis_desc_vc, tophit_desc_vc, how = "left", left_index = True, right_index = True).rename(columns={"count_x":"proksis_vc","count_y":"tophit_vc"})
most_prok_briteC["frac"] = most_prok_briteC["proksis_vc"] / most_prok_briteC["tophit_vc"]

In [47]:
# investigate TC numbers

In [48]:
tc_briteCs = list(tree_data_TC.desc.explode().value_counts().reset_index()["desc"])
        
filtered_dfs = {}
for briteC_id in tc_briteCs:
    filtered_df = tree_data_TC[tree_data_TC.desc.apply(lambda x: briteC_id in x)]
    filtered_dfs[briteC_id] = filtered_df
tcs_iter = iter(tc_briteCs)


In [49]:
tc_now = next(tcs_iter)
name, df = tc_now, summarize(filtered_dfs[tc_now], .3)
print(name)

Transporters [BR:ko02000]


In [130]:
# # sandbox
# kogs = set(module_data[module_data.CATEGORY_ID == "M00038"].KOGID) # glycolysis
# kogs = set(pathway_data[pathway_data.CATEGORY_ID == "map03010"].KOGID) # ribosome structure
# kogs = set(pathway_data[pathway_data.CATEGORY_ID == "map03060"].KOGID) # protein export
# kogs = set(pathway_data[pathway_data.CATEGORY_ID == "map04141"].KOGID) # ER
# kogs = set(pathway_data[pathway_data.CATEGORY_ID == "map04144"].KOGID) # Endo
# kogs = set(pathway_data[pathway_data.CATEGORY_ID == "map03022"].KOGID) # basal transcription factors
# kogs = set(pathway_data[pathway_data.CATEGORY_ID == "map04122"].KOGID) # sulfur relay system
# kogs = set(pathway_data[pathway_data.CATEGORY_ID == "map03050"].KOGID) # proteosome
# kogs = set(pathway_data[pathway_data.CATEGORY_ID == "map03018"].KOGID) # RNA degredation
# kogs = set(pathway_data[pathway_data.CATEGORY_ID == "map03440"].KOGID) # homologous recombination
# kogs = set(pathway_data[pathway_data.CATEGORY_ID == "map03420"].KOGID) # nucleotide excision repair

# trees_filtered = tree_data_tophit[(tree_data_tophit["kogid"].isin(kogs))].sort_values("kogid")
# trees_filtered = tree_data_tophit[tree_data_tophit["brite_C"].apply(lambda x: "03400" in x)].sort_values("kogid") # DNA repair/recomb
# trees_filtered = tree_data_tophit[(tree_data_tophit["c-ELW"] > .7) & (~tree_data_tophit.prok_taxa.isin(["Euryarchaeota", "TACK group", "Thermoplasmata", \
#                          "DPANN group"])) & (tree_data_tophit.euk_clade_size > 4)].reset_index().sort_values("kogid") # bact sister clade
# trees_cELW = trees_filtered[trees_filtered["c-ELW"].between(.5,.99)]
# systems = iter(trees_filtered.tree_name.unique())
# print(len(set(trees_filtered.tree_name.unique())), len(set(trees_cELW.tree_name)))
# microcosm_path = '/data/luojaa/eukgen/processing/microcosm_asgards/'

In [161]:
len(tree_data.euk_clade_rep.unique())


2964

In [171]:
tree_data_tophit

,tree_name,euk_clade_rep,euk_clade_size,euk_clade_weight,euk_leaf_clade,euk_LCA,euk_scope,euk_scope_len,prok_clade_rep,prok_clade_size,...,Target,Score,name,Pairwise_cov,kogid,brite_A,brite_B,brite_C,EC,TC
36262,TFG31145.1,QMYU01000226.1_2,29,1.000000,False,Asgard,Asgard,1,WP_148134982_1,5,...,K03303_prok,350.9,"lutP, lctP; lactate permease",0.926655,K03303,09180,09183,02000,none,2.A.14
117055,TFG26177.1,OLS12178.1,98,1.000000,False,Asgard,Asgard,1,WP_102101882_1,58,...,K03839_prok,155.4,"fldA, nifF, isiB; flavodoxin I",0.966216,K03839,09190,09191,99982,none,none
1083,RLI65667.1,JAHQWL010000096.1_52,52,1.000000,False,Asgard,Asgard,1,WP_014270848_1,3,...,K01596_prok,1317.7,"E4.1.1.32, pckA, PCK; phosphoenolpyruvate carboxykinase (GTP) [EC:4.1.1.32]",0.951768,K01596,09160|09150|09130|09100,09167|09101|09132|09155|09152,04152|04922|04964|04920|04151|04910|00010|04931|00620|03320|04068|00020,4.1.1.32,none
30014,RLI64519.1,UCH03867.1,47,1.000000,False,Asgard,Asgard,1,WP_137402020_1,210,...,K00290_euk,389.3,"LYS1; saccharopine dehydrogenase (NAD+, L-lysine forming) [EC:1.5.1.7]",0.794931,K00290,09100,09105,00310|00300,1.5.1.7,none
71966,RLI63982.1,JAHQWG010000035.1_122,30,1.000000,False,Asgard,Asgard,1,WP_179360697_1,16,...,K21636_prok,226.2,nrdD; ribonucleoside-triphosphate reductase (formate) [EC:1.1.98.6],0.845688,K21636,09100,09104,00240|00230,1.1.98.6,none
14486,PWI48493.1,JAEOTE010000053.1_25,25,0.961538,False,Asgard,Asgard,1,WP_014566100_1,7,...,K16422_prok,550.3,"hmo, nocN; 4-hydroxymandelate oxidase [EC:1.1.3.46]",0.951705,K16422,09100|09180,09110|09181|09109,00261|01008|01055,1.1.3.46,none
8612,OLS32638.1,JAHQWP010000086.1_24,48,1.000000,False,Asgard,Asgard,1,WP_012980906_1,65,...,K07072_prok,696.5,mfnF; (4-(4-[2-(gamma-L-glutamylamino)ethyl]phenoxymethyl)furan-2-yl)methanamine synthase [EC:2.5.1.131],0.942857,K07072,09100,09102,00680,2.5.1.131,none
4611,NHI93193.1,TFG30053.1,45,1.000000,False,Asgard,Asgard,1,WP_004633174_1,120,...,K00133_euk,846.3,asd; aspartate-semialdehyde dehydrogenase [EC:1.2.1.11],0.975000,K00133,09100,09105|09110,00261|00270|00260|00300,1.2.1.11,none
304,MBAA01000130.1_62,JAEOSY010000385.1_46,22,1.000000,False,Asgard,Asgard,1,WP_119070144_1,4,...,K17686_prok,1884.8,"copA, ctpA, ATP7; P-type Cu+ transporter [EC:7.2.2.8]",0.773774,K17686,09160|09150|09130,09176|09132|09154,04016|01524|04978,7.2.2.8,3.A.3.5|3.A.3.27
4332,LRSK01000052.1_12,UCE12298.1,125,1.000000,False,Asgard,Asgard,1,WP_103081089_1,3,...,K07138_prok,862.2,K07138; uncharacterized protein,0.912821,K07138,09190,09194,99997,none,none


In [173]:
tree_data_tophit.sort_values("c-ELW", ascending = False).drop_duplicates("euk_clade_rep").prok_taxa.value_counts()

prok_taxa
Bacillota                  7
Euryarchaeota              5
Actinomycetota             3
Thermodesulfobacteriota    2
FCB group                  2
TACK group                 1
PVC group                  1
Gammaproteobacteria        1
Cyanobacteriota            1
Spirochaetota              1
Mycoplasmatota             1
Name: count, dtype: int64

In [134]:
systems = iter(tree_data_tophit.tree_name.unique())
microcosm_path = '/data/luojaa/eukgen/processing/microcosm_asgards/'

In [155]:
#microcosm_path = '/data/tobiassonva/data/eukgen/processing/microcosm/'
system = next(systems)
#system = "EP01041P032404"
print(system)


data = tree_data_mapped.loc[[system]].sort_values(by=['Score', 'c-ELW'], ascending=False)
kogs, functions = data.Target, data.name
print(dict(zip(kogs, functions)))
    
clades = data.euk_clade_rep.unique()
print(f'System {system} has Eukaryotic clades {clades}')

clade = clades[0]

plot_data = data[data.euk_clade_rep == clade]

dist_bar = alt.Chart(data, title=f'{system}: {clade}').mark_bar().encode(
    x = alt.X('prok_taxa', sort=None, axis=alt.Axis(labelAngle=-45)),
    y = alt.Y('top_dist'),
    color = alt.Color('prok_taxa:O'),
    tooltip = alt.Tooltip(['top_dist', 'prok_clade_weight', 'prok_clade_size', 'prok_clade_rep', 'prok_leaf_clade'])
)

print(f'{microcosm_path}{system}/{system}.merged.fasta.muscle.treefile.annot')
tree = Tree(f'{microcosm_path}{system}/{system}.merged.fasta.muscle.treefile.annot')
tree.ladderize()
annot_tree, tree_img = color_tree(tree, view_in_notebook=True)


from core_functions.altair_plots import plot_alignment
aln_file = f'{microcosm_path}{system}/{system}.merged.fasta.muscle'

#leaf_names = [leaf.name for leaf in trees[system].get_leaves()]
leaf_names = [leaf.name for leaf in tree.get_leaves()]

plot, aln_data = plot_alignment(aln_file, seqlimit=100, plot_range=(0,300), label_order=leaf_names)

\
print(f'Tree has {len(leaf_names)} leaves')
display(dist_bar)
display(data.drop(["euk_clade_size","euk_clade_weight","euk_leaf_clade","euk_LCA", "euk_scope", "euk_scope_len","prok_clade_size","prok_clade_weight","prok_leaf_clade"], axis=1))#.iloc[:,[0,4,8,11,13,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31]])
#display(pd.DataFrame(CDD_annot.loc[system]))
display(plot)
display(tree_img)





StopIteration: 